<a href="https://colab.research.google.com/github/dhawalnegi1/EVA/blob/master/PROJECT-7/Assignment7_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all requirements

In [0]:
from keras import backend as K
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
np.random.seed(2017)

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation,SeparableConv2D,GlobalAveragePooling2D,Lambda
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.layers import Reshape,Input,Lambda
from keras.layers.merge import concatenate

Import the cifar dataset and store in the variables for training and test purpose.

In [15]:
# Load CIFAR10 Data
from keras.datasets import cifar10
(xtrain, ytrain), (xtest, ytest) = cifar10.load_data()
img_height, img_width, channel = xtrain.shape[1],xtrain.shape[2],xtrain.shape[3]
num_classes = len(np.unique(ytest))
print(num_classes)

10


Function accuracy is defined to calculate the validation accuracy by finding total no of images classfied correctly

In [0]:
def accuracy(xtest, ytest, model):
    result = model.predict(xtest)
    predicted_class = np.argmax(result, axis=1)
    true_class = np.argmax(ytest, axis=1)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

Pixel Normalisation is done for training and testing data. And the class labels for training and testing data is converted into one-hot encoding.

In [0]:
xtrain = xtrain.astype('float32')/255
xtest = xtest.astype('float32')/255
# convert class labels to binary class labels
ytrain = np_utils.to_categorical(ytrain, num_classes)
ytest = np_utils.to_categorical(ytest, num_classes)

In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

##Block1 - Normal Convolution

2 layers of normal 3x3 convolution is add and after that bottleneck layer of 1x1 and maxpooling is used.

In [7]:
inp = Input(shape=(img_height, img_width, channel,))

L1 = SeparableConv2D(32, (3,3), strides=(1,1), use_bias=False,border_mode='same')(inp)
L1 = BatchNormalization()(L1)
L1 = Activation('relu')(L1)
L1 = SeparableConv2D(64, (3,3), strides=(1,1), use_bias=False,border_mode='same')(L1)
L1 = BatchNormalization()(L1)
L1 = Activation('relu')(L1)

L2 = Conv2D(32, (3,3), strides=(1,1), use_bias=False,border_mode='same')(L1)
L2 = BatchNormalization()(L2)
L2 = Activation('relu')(L2)
L2 = Conv2D(64, (3,3), strides=(1,1), use_bias=False,border_mode='same')(L2)
L2 = BatchNormalization()(L2)
L2 = Activation('relu')(L2)

L3 = Conv2D(32, (3,3), strides=(1,1), use_bias=False,border_mode='same')(L2)
L3 = BatchNormalization()(L3)
L3 = Activation('relu')(L3)
L3 = Conv2D(64, (3,3), strides=(1,1), use_bias=False,border_mode='same')(L3)
L3 = BatchNormalization()(L3)
L3 = Activation('relu')(L3)

C1 = concatenate([L1,L3])

L4 = SeparableConv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C1)
L4 = BatchNormalization()(L4)
L4 = Activation('relu')(L4)
L4 = SeparableConv2D(64, (3,3), strides=(1,1), use_bias=False,padding='same')(L4)
L4 = BatchNormalization()(L4)
L4 = Activation('relu')(L4)

C2 = concatenate([L1,L4])

MAX1 = MaxPooling2D(pool_size=(2, 2))(C2)

L5 = SeparableConv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(MAX1)
L5 = BatchNormalization()(L5)
L5 = Activation('relu')(L5)

l1= Lambda(space_to_depth_x2)(L1)
l4= Lambda(space_to_depth_x2)(L4)
C2 = concatenate([l1,l4,L5])

L6 = Conv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C2)
L6 = BatchNormalization()(L6)
L6 = Activation('relu')(L6)
L6 = Conv2D(64, (3,3), strides=(1,1), use_bias=False,padding='same')(L6)
L6 = BatchNormalization()(L6)
L6 = Activation('relu')(L6)

l3= Lambda(space_to_depth_x2)(L3)
l2= Lambda(space_to_depth_x2)(L2)
C3 = concatenate([l3,l4,L5,L6])

L7 = SeparableConv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C3)
L7 = BatchNormalization()(L7)
L7 = Activation('relu')(L7)

C3 = concatenate([l1,l3,l4,L5,L6,L7])

L8 = SeparableConv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C3)
L8 = BatchNormalization()(L8)
L8 = Activation('relu')(L8)
L8 = SeparableConv2D(64, (3,3), strides=(1,1), use_bias=False,padding='same')(L8)
L8 = BatchNormalization()(L8)
L8 = Activation('relu')(L8)

C4 = concatenate([l1,l4,L5,L7,L8])

MAX2 = MaxPooling2D(pool_size=(2, 2))(C4)

l6= Lambda(space_to_depth_x2)(L6)
C5 = concatenate([l6,MAX2])

L9 = Conv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C5)
L9 = BatchNormalization()(L9)
L9 = Activation('relu')(L9)
L9 = Conv2D(64, (3,3), strides=(1,1), use_bias=False,padding='same')(L9)
L9 = BatchNormalization()(L9)
L9 = Activation('relu')(L9)

l1= Lambda(space_to_depth_x2)(l1)
l2= Lambda(space_to_depth_x2)(l2)
l4= Lambda(space_to_depth_x2)(l4)
l7= Lambda(space_to_depth_x2)(L7)
C6 = concatenate([l1,l2,l4,l7,L9])

L10 = SeparableConv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C6)
L10 = BatchNormalization()(L10)
L10 = Activation('relu')(L10)
L10 = SeparableConv2D(64, (3,3), strides=(1,1), use_bias=False,padding='same')(L10)
L10 = BatchNormalization()(L10)
L10 = Activation('relu')(L10)

l3= Lambda(space_to_depth_x2)(l3)
l5= Lambda(space_to_depth_x2)(L5)
C7 = concatenate([l2,l3,l5,L9,L10])

L11 = Conv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C7)
L11 = BatchNormalization()(L11)
L11 = Activation('relu')(L11)
L11 = Conv2D(64, (3,3), strides=(1,1), use_bias=False,padding='same')(L11)
L11 = BatchNormalization()(L11)
L11 = Activation('relu')(L11)

C8 = concatenate([l3,l4,l5,l7,L10,L11])

L12 = SeparableConv2D(32, (3,3), strides=(1,1), use_bias=False,padding='same')(C8)
L12 = BatchNormalization()(L12)
L12 = Activation('relu')(L12)
L12 = SeparableConv2D(64, (3,3), strides=(1,1), use_bias=False,padding='same')(L12)
L12 = BatchNormalization()(L12)
L12 = Activation('relu')(L12)

C9 = concatenate([l4,l7,L10,L12])

W0618 17:12:50.094331 139656125454208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0618 17:12:50.130024 139656125454208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (3, 3), strides=(1, 1), use_bias=False, padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
W0618 17:12:50.139378 139656125454208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform in

Finally the no of channels are reduced to 10 and flatten and softmax is applied.

In [8]:
OUTPUT = Conv2D(10,8)(C9)
print(OUTPUT)
OUTPUT = GlobalAveragePooling2D()(OUTPUT)
print(OUTPUT)
Output = Activation('softmax')(OUTPUT)

Tensor("conv2d_11/BiasAdd:0", shape=(?, 1, 1, 10), dtype=float32)
Tensor("global_average_pooling2d_1/Mean:0", shape=(?, 10), dtype=float32)


In [9]:
model= Model(inputs=[inp],outputs=[Output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 32, 32, 32)   123         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         separable_conv2d_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
separable_

Model is compiled and summery is printed.

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

W0618 17:12:55.402479 139656125454208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Model is Trained for 50 epocs and checkpoint is used to saved the model with best validation accuracy and time taken to run is calculated. 

In [11]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(zoom_range=0.0, 
                             horizontal_flip=False)

from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
# train the model
start = time.time()
# Train the model
filepath="saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model_info = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size = 128),
                                 samples_per_epoch = xtrain.shape[0], nb_epoch = 100, 
                                 validation_data = (xtest,ytest),callbacks= callbacks_list, verbose=1)
end = time.time()
print ("Model took %0.2f seconds to train"%(end - start))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., verbose=1, steps_per_epoch=390, epochs=100)`
W0618 17:12:55.535671 139656125454208 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a futur

Epoch 1/100
390/390 [==============================] - 72s 184ms/step - loss: 7.8231 - acc: 0.2362 - val_loss: 10.4262 - val_acc: 0.1774

Epoch 00001: val_acc improved from -inf to 0.17740, saving model to saved.hdf5
Epoch 2/100
390/390 [==============================] - 64s 163ms/step - loss: 1.7076 - acc: 0.5201 - val_loss: 11.4586 - val_acc: 0.1479

Epoch 00002: val_acc did not improve from 0.17740
Epoch 3/100
390/390 [==============================] - 63s 161ms/step - loss: 1.5894 - acc: 0.6081 - val_loss: 2.6543 - val_acc: 0.5526

Epoch 00003: val_acc improved from 0.17740 to 0.55260, saving model to saved.hdf5
Epoch 4/100
390/390 [==============================] - 63s 161ms/step - loss: 1.1634 - acc: 0.6737 - val_loss: 1.1710 - val_acc: 0.6638

Epoch 00004: val_acc improved from 0.55260 to 0.66380, saving model to saved.hdf5
Epoch 5/100
390/390 [==============================] - 63s 161ms/step - loss: 0.9799 - acc: 0.7189 - val_loss: 1.1665 - val_acc: 0.6801

Epoch 00005: val_acc

In [23]:
score = model.evaluate(xtest, ytest, verbose=1)
print(score)

10000/10000 [==============================] - 7s 662us/step
[2.3429378987312317, 0.7285]
